In [1]:
#from numpy import array, zeros, exp, random, dot, shape, reshape, meshgrid, linspace
import numpy as np

import matplotlib.pyplot as plt # for plotting
import matplotlib
matplotlib.rcParams['figure.dpi']=300 # highres display

# for subplots within subplots:
from matplotlib import gridspec

# for nice inset colorbars: (approach changed from lecture 1 'Visualization' notebook)
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition

# for updating display 
# (very simple animation)
from IPython.display import clear_output
from time import sleep

In [ ]:
class NN:

    # backpropagation and training routines

    # this is basically a merger of the backpropagation
    # code shown in lecture 2 and some of the 
    # visualization code used in the lecture 1 tutorials!

    def __init__(self, nLayers, nBatch, layerSize, useRELU = False, wMax = 1, bMax = 1):
        self.nlayers = nLayers
        self.batchsize = nBatch
        self.layersize = layerSize
        self.useRELU = useRELU
        
        #Initialise the weights and biases with random numbers
        self.w=[np.random.uniform(low = -1*wMax, high =+ 1*wMax, size = [self.layersize[j], self.layersize[j+1] ]) for j in range(self.nlayers)]
        self.b=[np.random.uniform(low = -1*bMax, high =+ 1*bMax, size = [self.layersize[j+1]]) for j in range(self.nlayers)]
        
        #Define the arrays for various outputs needed for backpropagation
        self.yArray = [np.zeros([self.batchsize, self.layersize[j]]) for j in range(self.nlayers + 1)]
        self.dfArray = [np.zeros([self.batchsize, self.layersize[j+1]]) for j in range(self.nlayers)]
        self.dwArray = [np.zeros([self.layersize[j], self.layersize[j+1]]) for j in range(self.nlayers)]
        self.dbArray = [np.zeros(self.layersize[j+1]) for j in range(self.nlayers)]

    def net_f_df(self, z, activation):
        # return both value f(z) and derivative f'(z)
        if activation=='sigmoid':
            return([1/(1+np.exp(-z)), 
                    1/((1+np.exp(-z))*(1+np.exp(z))) ])
        elif activation=='jump': # cheating a bit here: replacing f'(z)=delta(z) by something smooth
            return([np.array(z>0,dtype='float'), 
                    10.0/((1+np.exp(-10*z))*(1+np.exp(10*z))) ] )
        elif activation=='linear':
            return([z,
                    1.0])
        elif activation=='reLU':
            return([(z>0)*z,
                    (z>0)*1.0
                ])

    def forward_step(self,y,w,b,activation):
        """
        Go from one layer to the next, given a 
        weight matrix w (shape [n_neurons_in,n_neurons_out])
        a bias vector b (length n_neurons_out)
        and the values of input neurons y_in 
        (shape [batchsize,n_neurons_in])
        
        returns the values of the output neurons in the next layer 
        (shape [batchsize, n_neurons_out])
        """    
        # calculate values in next layer, from input y
        z=np.dot(y,w)+b # w=weights, b=bias vector for next layer
        return(self.net_f_df(z,activation)) # apply nonlinearity and return result

    def apply_net(self,x_in): # one forward pass through the network
        global Weights, Biases, NumLayers, Activations
        global y_layer, df_layer # for storing y-values and df/dz values
        
        y=np.copy(x_in) # start with input values
        y_layer[0]=np.copy(y)
        for j in range(NumLayers): # loop through all layers
            # j=0 corresponds to the first layer above the input
            y,df=self.forward_step(y,Weights[j],Biases[j],Activations[j]) # one step
            df_layer[j]=np.copy(df) # store f'(z) [needed later in backprop]
            y_layer[j+1]=np.copy(y) # store f(z) [also needed in backprop]        
        return(y)

    def apply_net_simple(self,x_in): # one forward pass through the network
        # no storage for backprop (this is used for simple tests)
        global Weights, Biases, NumLayers, Activations
        
        y=x_in # start with input values
        for j in range(NumLayers): # loop through all layers
            # j=0 corresponds to the first layer above the input
            y,df=self.forward_step(y,Weights[j],Biases[j],Activations[j]) # one step
        return(y)

    def backward_step(self,delta,w,df): 
        # delta at layer N, of batchsize x layersize(N))
        # w between N-1 and N [layersize(N-1) x layersize(N) matrix]
        # df = df/dz at layer N-1, of batchsize x layersize(N-1)
        return( np.dot(delta,np.transpose(w))*df )

    def backprop(self,y_target): # one backward pass through the network
        # the result will be the 'dw_layer' matrices that contain
        # the derivatives of the cost function with respect to
        # the corresponding weight
        global y_layer, df_layer, Weights, Biases, NumLayers
        global dw_layer, db_layer # dCost/dw and dCost/db (w,b=weights,biases)

        batchsize=np.shape(y_target)[0]
        delta=(y_layer[-1]-y_target)*df_layer[-1]
        dw_layer[-1]=np.dot(np.transpose(y_layer[-2]),delta)/batchsize
        db_layer[-1]=delta.sum(0)/batchsize
        for j in range(NumLayers-1):
            delta=self.backward_step(delta,Weights[-1-j],df_layer[-2-j])
            dw_layer[-2-j]=np.dot(np.transpose(y_layer[-3-j]),delta)/batchsize # batchsize was missing in old code?
            db_layer[-2-j]=delta.sum(0)/batchsize
            
    def gradient_step(self,eta): # update weights & biases (after backprop!)
        global dw_layer, db_layer, Weights, Biases
        
        for j in range(NumLayers):
            Weights[j]-=eta*dw_layer[j]
            Biases[j]-=eta*db_layer[j]
            
    def train_net(self,x_in,y_target,eta): # one full training batch
        # x_in is an array of size batchsize x (input-layer-size)
        # y_target is an array of size batchsize x (output-layer-size)
        # eta is the stepsize for the gradient descent
        global y_out_result
        
        y_out_result=self.apply_net(x_in)
        self.backprop(y_target)
        self.gradient_step(eta)
        cost=0.5*((y_target-y_out_result)**2).sum()/np.shape(x_in)[0]
        return(cost)

    def init_layer_variables(self,weights,biases,activations):
        global Weights, Biases, NumLayers, Activations
        global LayerSizes, y_layer, df_layer, dw_layer, db_layer

        Weights=weights
        Biases=biases
        Activations=activations
        NumLayers=len(Weights)

        LayerSizes=[2]
        for j in range(NumLayers):
            LayerSizes.append(len(Biases[j]))

        y_layer=[[] for j in range(NumLayers+1)]
        df_layer=[[] for j in range(NumLayers)]
        dw_layer=[np.zeros([LayerSizes[j],LayerSizes[j+1]]) for j in range(NumLayers)]
        db_layer=[np.zeros(LayerSizes[j+1]) for j in range(NumLayers)]